# Resampling observational noise

First we need some imports.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy import stats
import pylab as plt
import torch
import torchist
import swyft

## Training data

Now we generate training data.  As simple example, we consider the model

$$
x = z + \epsilon
$$

where the parameter $z \sim \mathcal{N}(\mu = 0, \sigma = 1)$ is standard normal distributed, and $\epsilon \sim \mathcal{N}(\mu = 0, \sigma = 0.1)$ is a small noise contribution.  We are interested in the posterior of $z$ given a measurement of parameter $x$.

In [3]:
class Simulator(swyft.Simulator):
    def __init__(self):
        super().__init__()
        self.transform_samples = swyft.to_numpy32

    def forward(self, trace):
        z = trace.sample('z', lambda: np.random.rand(1))
        x = trace.sample('x', lambda z: z + np.random.randn(1)*0.1, z)
        
sim = Simulator()

In [4]:
samples = sim.sample(500)

100%|██████████| 500/500 [00:00<00:00, 32344.49it/s]


In [5]:
resampler = sim.get_resampler(targets = ["x"])

In [6]:
class Network(swyft.SwyftModule):
    def __init__(self):
        super().__init__()
        self.logratios = swyft.LogRatioEstimator_1dim(num_features = 1, num_params = 1, varnames = 'z')

    def forward(self, A, B):
        logratios = self.logratios(A['x'], B['z'])
        return logratios

## Trainer

Training is now done using the `SwyftTrainer` class, which extends `pytorch_lightning.Trainer` by methods like `infer` (see below).

In [7]:
trainer = swyft.SwyftTrainer(accelerator = 'gpu', max_epochs = 20, precision = 32)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


The `swyft.Samples` class provides convenience functions to generate data loaders for training and validation data.

In [8]:
dl_train = samples[:-100].get_dataloader(batch_size = 16, shuffle = True, on_after_load_sample = resampler)
dl_valid = samples[-100:].get_dataloader(batch_size = 16)

In [9]:
network = Network()

In [10]:
trainer.fit(network, dl_train, dl_valid)

/home/weniger/miniconda3b/envs/zero/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/weniger/codes/swyft/notebooks/lightning_logs/version_10034687/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | logratios | LogRatioEstimator_1dim | 17.4 K
-----------------------------------------------------
17.4 K    Trainable params
0         Non-trainable params
17.4 K    Total params
0.070     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/weniger/miniconda3b/envs/zero/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/weniger/miniconda3b/envs/zero/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/weniger/miniconda3b/envs/zero/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1894: PossibleUserWarning: The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  78%|███████▊  | 25/32 [00:00<00:00, 102.46it/s, loss=-0.509, v_num=1e+7]
Validation: 0it [00:00, ?it/s]
Epoch 1:  78%|███████▊  | 25/32 [00:00<00:00, 100.92it/s, loss=-0.555, v_num=1e+7, val_loss=-.0479]
Validation: 0it [00:00, ?it/s]
Epoch 2:  78%|███████▊  | 25/32 [00:00<00:00, 96.04it/s, loss=-0.604, v_num=1e+7, val_loss=-.384] 
Validation: 0it [00:00, ?it/s]
Epoch 3:  78%|███████▊  | 25/32 [00:00<00:00, 95.48it/s, loss=-0.544, v_num=1e+7, val_loss=-.529] 
Validation: 0it [00:00, ?it/s]
Epoch 4:  78%|███████▊  | 25/32 [00:00<00:00, 96.38it/s, loss=-0.57, v_num=1e+7, val_loss=-.559]  
Validation: 0it [00:00, ?it/s]
Epoch 5:  78%|███████▊  | 25/32 [00:00<00:00, 97.14it/s, loss=-0.572, v_num=1e+7, val_loss=-.556]
Validation: 0it [00:00, ?it/s]
Epoch 6:  78%|███████▊  | 25/32 [00:00<00:00, 88.67it/s, loss=-0.573, v_num=1e+7, val_loss=-.538] 
Validation: 0it [00:00, ?it/s]
Epoch 7:  78%|███████▊  | 25/32 [00:00<00:00, 100.38it/s, loss=-0.543, v_num=1e+7, val_loss=-.553]
Validat

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 32/32 [00:00<00:00, 97.59it/s, loss=-0.599, v_num=1e+7, val_loss=-.554] 
